In [1]:
 import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-03 19:31:21--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.51MB/s    in 0.2s    

2022-01-03 19:31:22 (5.51 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Home_v1_00.tsv.gz'), sep=r'\t', header=True, inferSchema=True)

# Show DataFrame
df.head()

Row(marketplace='US', customer_id=33670092, review_id='R1UUISQ1GKOJTI', product_id='B00EE62UAE', product_parent=583436067, product_title='Trademark Home Portable Closet, White', product_category='Home', star_rating=1, helpful_votes=0, total_votes=0, vine='N', verified_purchase='Y', review_headline='Run', review_body="Don't buy it clapse on me within in 5 minutes", review_date='2015-08-31')

In [ ]:
#Count the number of records (rows) in the dataset.
df.count()


6221559

In [5]:
# check for duplicate records
df.distinct().count()

6221559

Transform the dataset to fit the tables in the schema file. Be sure the DataFrames match in data type and in column name.

In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
#create review_id_table table
from pyspark.sql.types import StringType, BooleanType, IntegerType, DateType
  
review_id_table = df.select(
    df.review_id, df.customer_id,df.product_id,df.product_parent, df.review_date.cast(DateType())
    )

review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
#create products table

products = df.select(
    df.product_id, df.product_title
    )

products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [ ]:
#create customers table

customers = df.groupBy(df.customer_id).count().withColumnRenamed("count","customer_count")

customers.printSchema()
customers.orderBy(customers.customer_count, ascending = False).show()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   32405532|          3515|
|   20266780|           424|
|   53084107|           415|
|   16923020|           378|
|   52868156|           358|
|   51346302|           322|
|   48428870|           315|
|   12609448|           289|
|   47501917|           273|
|   52833354|           272|
|   35178127|           242|
|   12380650|           233|
|   11438572|           227|
|   26904884|           225|
|    3200008|           220|
|   52639627|           219|
|   51148531|           210|
|   17001126|           207|
|   25993756|           206|
|    8619660|           205|
+-----------+--------------+
only showing top 20 rows



In [ ]:
#create vine table

vine_table = df.select(
    df.review_id, df.star_rating, df.helpful_votes, df.total_votes, df.vine
    )

vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



Load the DataFrames that correspond to tables into an RDS instance. Note: This process can take up to 10 minutes for each. Be sure that everything is correct before uploading.

In [ ]:
# Configure settings for RDS
from config import password, endpoint 

mode = "append"
jdbc_url="jdbc:postgresql://"+endpoint+":5432/amazon_reviews"
configuration = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

In [ ]:
 # Write DataFrame to active_user table in RDS

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=configuration)

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=configuration)

In [ ]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=configuration)

In [ ]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=configuration)